 ## Install the Required Packages

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install diffusers==0.4.1
!pip install accelerate
!pip install torchvision
!pip install transformers>=4.21.0
!pip install ftfy
!pip install tensorboard
!pip install modelcards
!pip install datasets
!pip install -U diffusers
!pip install h5py
!pip install mat4py

  Cloning https://github.com/huggingface/diffusers.git to /state/partition1/job-27956462/pip-req-build-iloef8ne
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /state/partition1/job-27956462/pip-req-build-iloef8ne
  Resolved https://github.com/huggingface/diffusers.git to commit 6b68afd8e4cbf05a06a9ed72127bfdc3acff1db4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 6.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 74.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 14.1 MB/s

## Import the Required Packages

In [ ]:
import datasets
import shutil
import h5py
import numpy as np
import io
import PIL.Image as Image
from torchvision.utils import save_image
import torch
import json
from datasets import load_dataset
import pickle
import torch
from diffusers import StableDiffusionPipeline
from mat4py import loadmat

Check GPU Information

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Initialize the accelerate environment

In [ ]:
!accelerate config default

Configuration already exists at /home/sv2128/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


## Hugging Face Hub Login

You need to accept the model license before downloading or using the weights. In this example we’ll use model version v1-4, so you’ll need to visit its card (https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree.

You have to be a registered user in 🤗 Hugging Face Hub, and you’ll also need to use an access token for the code to work. For more information on access tokens, please refer to this section of the documentation (https://huggingface.co/docs/hub/security-tokens).

In [ ]:
from huggingface_hub import login
login(token='your_access_token')

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /home/sv2128/.huggingface/token
Login successful


## Get Set ID for Flower Dataset

Load the mat file from https://www.robots.ox.ac.uk/~vgg/data/flowers/102/

It contains ids of images across train test and valid splits

In [ ]:
data = loadmat('path/to/set/mat/file')

#print(data['trnid'])
print(len(data['trnid']))
#print(data['valid'])
print(len(data['valid']))
#print(data['tstid'])
print(len(data['tstid']))

1020
1020
6149


## Get Labels for Flower Dataset

Load the mat file from https://www.robots.ox.ac.uk/~vgg/data/flowers/102/

It contains labels of all the images

In [ ]:
data_labels = loadmat('path/to/label/mat/file')
#print(data_labels)
print(len(data_labels['labels']))

8189


In [ ]:
train_image_to_label={}
for i in data['trnid']:
  train_image_to_label[i]=data_labels['labels'][i-1]
valid_image_to_label={}
for i in data['valid']:
  valid_image_to_label[i]=data_labels['labels'][i-1]
test_image_to_label={}
for i in data['tstid']:
  test_image_to_label[i]=data_labels['labels'][i-1]

#print(train_image_to_label)
print(len(train_image_to_label))
#print(valid_image_to_label)
print(len(valid_image_to_label))
#print(test_image_to_label)
print(len(test_image_to_label))

1020
1020
6149


## Function to Construct Oxford-102 (Flower) Dataset as Required by Stable Diffusion

To run on your own training files you need to prepare the dataset according to the format required by datasets. You can upload your dataset to the Hugging Face Hub, or you can prepare a local folder with your files. This documentation (https://huggingface.co/docs/hub/security-tokens) explains how to do it.

Make sure there is an empty train folder in FLOWER_DIR

Keep flowers.hdf5 file (https://drive.google.com/file/d/0B0ywwgffWnLLcms2WWJQRFNSWXM/view?resourcekey=0-Av8zFbeDDvNcF1sSjDR32w) in FLOWER_DIR. This file contains captions for all the images.

Note: The split of data across train/valid/test is not same for the classification task on Oxford-102 (Flower) and image captioning task. We arrange the data in image captioning task according to the splits of classification task.

In [ ]:
"""

This funtion uses only the training and validation splits according to classification 
task of Oxford-102 (Flower) dataset and converts them into the required format 
for fine-tuning of the Stable Diffusion model.

"""

def construct_flower(FLOWER_DIR):

  filename = FLOWER_DIR+"/flowers.hdf5"

  f=h5py.File(filename, "r")
  metajson_list=[]
  count=0
  for i in list(f['train']):

    
    temp_meta={}
    
    name=np.array(f['train'][i]['name']).astype(str)
    name=np.array_str(name)
    #print(name)
    name=name.split("_")[1]
    name=name.lstrip("0")
    #print("Name: "+name)
    name=int(name)
    
    
    if name in train_image_to_label.keys() or name in valid_image_to_label.keys():
        
        count=count+1
        caption=np.array(f['train'][i]['txt']).astype(str)
        caption=np.array_str(caption)
        label=np.array(f['train'][i]['class']).astype(str)
        label=np.array_str(label)
        label=label.split("_")[1]
        label=label.lstrip("0")
        #print("Label: "+label)
        #image_name=i
        image=bytes(np.array(f['train'][i]['img']))
        image = Image.open(io.BytesIO(image))
        image.save(FLOWER_DIR+'/train/'+str(name)+"_"+label+"_"+str(count)+'.png')
        temp_meta['file_name']=str(name)+"_"+label+"_"+str(count)+'.png'
        temp_meta['text']=caption
        temp_meta['label']=label
        metajson_list.append(temp_meta)
        
  for i in list(f['valid']):

    
    temp_meta={}

    name=np.array(f['valid'][i]['name']).astype(str)
    name=np.array_str(name)
    #print(name)
    name=name.split("_")[1]
    name=name.lstrip("0")
    #print("Name: "+name)
    name=int(name)


    if name in train_image_to_label.keys() or name in valid_image_to_label.keys():

        count=count+1
        caption=np.array(f['valid'][i]['txt']).astype(str)
        caption=np.array_str(caption)
        label=np.array(f['valid'][i]['class']).astype(str)
        label=np.array_str(label)
        label=label.split("_")[1]
        label=label.lstrip("0")
        #print("Label: "+label)
        #image_name=i
        image=bytes(np.array(f['valid'][i]['img']))
        image = Image.open(io.BytesIO(image))
        image.save(FLOWER_DIR+'/train/'+str(name)+"_"+label+"_"+str(count)+'.png')
        temp_meta['file_name']=str(name)+"_"+label+"_"+str(count)+'.png'
        temp_meta['text']=caption
        temp_meta['label']=label
        metajson_list.append(temp_meta)

  for i in list(f['test']):

    
    temp_meta={}
    
    name=np.array(f['test'][i]['name']).astype(str)
    name=np.array_str(name)
    #print(name)
    name=name.split("_")[1]
    name=name.lstrip("0")
    #print("Name: "+name)
    name=int(name)
    
    
    if name in train_image_to_label.keys() or name in valid_image_to_label.keys():
        
        count=count+1
        caption=np.array(f['test'][i]['txt']).astype(str)
        caption=np.array_str(caption)
        label=np.array(f['test'][i]['class']).astype(str)
        label=np.array_str(label)
        label=label.split("_")[1]
        label=label.lstrip("0")
        #print("Label: "+label)
        #image_name=i
        image=bytes(np.array(f['test'][i]['img']))
        image = Image.open(io.BytesIO(image))
        image.save(FLOWER_DIR+'/train/'+str(name)+"_"+label+"_"+str(count)+'.png')
        temp_meta['file_name']=str(name)+"_"+label+"_"+str(count)+'.png'
        temp_meta['text']=caption
        temp_meta['label']=label
        metajson_list.append(temp_meta)

In [ ]:
FLOWER_DIR = "path/to/FLOWER_DIR"

In [ ]:
construct_flower(FLOWER_DIR)

{'file_name': '17_77_1.png', 'text': 'this flower has thick pale green petals under thick purple and white fringe.\n', 'label': '77'}
{'file_name': '17_77_2.png', 'text': "this flower's leaves appear to be large and pointy at the ends. a burnt orange color makes it exquisite to look at. the stamen is yellow in color, easy to see, and easy to get to for pollination. the pollen tube is green in color and long, sticking up above flower. the whole pistil is green in color and easy to see and get to.\n", 'label': '77'}
{'file_name': '17_77_3.png', 'text': 'this flower has white petals with blue and white filaments arranged in a flat disk formation.\n', 'label': '77'}
{'file_name': '17_77_4.png', 'text': 'this unique white flower has lots of thin blue petals with a dark green center and purple lines.\n', 'label': '77'}
{'file_name': '17_77_5.png', 'text': "this flower's leaves appear to be large and pointy at the ends. a burnt orange color makes it exquisite to look at. the stamen is yellow 

## Fine-tune Stable Diffusion

The fine-tuning of Stable Diffusion takes around 3 hours on V100 GPU for epochs. Please tune the hyper-parameters based on your requirements.

In [ ]:
!(accelerate launch HF_StableD_CV/train_text_to_image.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" \
  --train_data_dir="path/to/FLOWER_DIR/train" \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=16 \
  --gradient_accumulation_steps=8 \
  --gradient_checkpointing \
  --mixed_precision="bf16" \
  --num_train_epochs=5 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="path/to/FLOWER_DIR/output/directory" \
)

/ext3/miniconda3/envs/ml/lib/python3.10/site-packages/diffusers/configuration_utils.py:195: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddpm.DDPMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
Resolving data files: 100%|████████████| 10201/10201 [00:00<00:00, 22865.53it/s]
12/09/2022 20:41:59 - WARNING - datasets.builder - Using custom data configuration default-395ace24d8d245bd










































































































Extracting data files: 0it [00:00, ?it/s]
Dataset imagefolder downloaded and prepared to /home/sv2128/.cache/huggingface/datasets/imagefolder/default-395ace24d8d245bd/0.0.0/37f

## Visualize Epoch Vs Loss Graph 

In [ ]:
!tensorboard --logdir path/to/FLOWER_DIR/output/directory


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


## Function to Generate Oxford-102 (Flower) Dataset

This function generates images based on the labels as text prompts present in test set of Oxford-102 (Flower)

Make sure there is an empty test folder in FLOWER_DIR location

In [ ]:
def generate_flower(FLOWER_DIR):

  filename = FLOWER_DIR+"/flowers.hdf5"
  f=h5py.File(filename, "r")
  model_path = FLOWER_DIR+"path/to/FLOWER_DIR/output/directory"
  pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
  pipe.safety_checker = lambda images, clip_input: (images, False)
  pipe.to("cuda")

  count=0
  temp=1
  for i in list(f['train']):
    
    name=np.array(f['train'][i]['name']).astype(str)
    name=np.array_str(name)
    #print(name)
    name=name.split("_")[1]
    name=name.lstrip("0")
    #print("Name: "+name)
    name=int(name)
    
    #print("Temp: " + str(temp))
    if name in test_image_to_label.keys():
        
        if temp!=1:
            temp=temp-1
        else:
            temp=5
            count=count+1
            print(count)
            caption=np.array(f['train'][i]['txt']).astype(str)
            caption=np.array_str(caption)
            label=np.array(f['train'][i]['class']).astype(str)
            label=np.array_str(label)
            label=label.split("_")[1]
            label=label.lstrip("0")
            #print("Label: "+label)
            image = pipe(prompt=caption).images[0]
            image.save(FLOWER_DIR+'/test/'+str(name)+"_"+label+"_"+str(count)+'.png')
  temp=1
  for i in list(f['valid']):

    name=np.array(f['valid'][i]['name']).astype(str)
    name=np.array_str(name)
    #print(name)
    name=name.split("_")[1]
    name=name.lstrip("0")
    #print("Name: "+name)
    name=int(name)
    

    if name in test_image_to_label.keys():
        
        if temp!=1:
            temp=temp-1
        else:
            temp=5
            count=count+1
            print(count)
            caption=np.array(f['valid'][i]['txt']).astype(str)
            caption=np.array_str(caption)
            label=np.array(f['valid'][i]['class']).astype(str)
            label=np.array_str(label)
            label=label.split("_")[1]
            label=label.lstrip("0")
            #print("Label: "+label)
            image = pipe(prompt=caption).images[0]
            image.save(FLOWER_DIR+'/test/'+str(name)+"_"+label+"_"+str(count)+'.png')

  temp=1
  for i in list(f['test']):

    
    name=np.array(f['test'][i]['name']).astype(str)
    name=np.array_str(name)
    #print(name)
    name=name.split("_")[1]
    name=name.lstrip("0")
    #print("Name: "+name)
    name=int(name)
    
    
    if name in test_image_to_label.keys():
        
        if temp!=1:
            temp=temp-1
        else:
            temp=5
        
            count=count+1
            print(count)
            caption=np.array(f['test'][i]['txt']).astype(str)
            caption=np.array_str(caption)
            label=np.array(f['test'][i]['class']).astype(str)
            label=np.array_str(label)
            label=label.split("_")[1]
            label=label.lstrip("0")
            #print("Label: "+label)
            image = pipe(prompt=caption).images[0]
            image.save(FLOWER_DIR+'/test/'+str(name)+"_"+label+"_"+str(count)+'.png')
  print(count) 


In [ ]:
generate_flower(FLOWER_DIR)

1


100%|██████████| 50/50 [00:02<00:00, 24.54it/s]


2


100%|██████████| 50/50 [00:02<00:00, 24.82it/s]


3


...

100%|██████████| 50/50 [00:01<00:00, 25.08it/s]


386


100%|██████████| 50/50 [00:02<00:00, 24.07it/s]


387


 46%|████▌     | 23/50 [00:00<00:01, 25.37it/s]